In [1]:
# Install packages
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow==2.18.0
!pip install opencv-python

ERROR: Could not find a version that satisfies the requirement tensorflow.compat.v1 (from versions: none)
ERROR: No matching distribution found for tensorflow.compat.v1


In [2]:
# Run imports
import os
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import PIL
import pandas as pd
import tensorflow
import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tqdm
import absl.logging

tf.get_logger().setLevel('ERROR')
absl.logging.set_verbosity(absl.logging.ERROR)
mpl.rcParams.update({
    'font.size': 10,
})

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DATASET_PATH = '/content/drive/MyDrive/dataset'
TRAIN_DIR = os.path.join(DATASET_PATH, 'training')
TEST_DIR = os.path.join(DATASET_PATH, 'testing')

In [5]:
def get_video_paths_and_labels(base_path):
    paths = []
    labels = []
    label_map = {'passive': 0, 'active': 1}

    for label_name, label_value in label_map.items():
        folder = os.path.join(base_path, label_name)
        for fname in os.listdir(folder):
            if fname.endswith('.mp4'):
                paths.append(os.path.join(folder, fname))
                labels.append(label_value)

    return paths, labels

train_paths, train_labels = get_video_paths_and_labels('/content/drive/MyDrive/dataset/training')
test_paths, test_labels = get_video_paths_and_labels('/content/drive/MyDrive/dataset/testing')


In [6]:
# Paramètres
frame_stride = 2
num_frames = 16
resolution = 224
num_classes = 2
batch_size = 4

# Fonction utilisée via tf.py_function
def read_video_opencv(path):
    path = path.decode() if isinstance(path, bytes) else path
    cap = cv2.VideoCapture(path)
    frames = []
    success = True

    while success:
        success, frame = cap.read()
        if success:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)
    cap.release()

    frames = frames[::frame_stride]
    frames = [cv2.resize(f, (resolution, resolution)) for f in frames]

    if len(frames) >= num_frames:
        frames = frames[:num_frames]
    else:
        pad = [np.zeros((resolution, resolution, 3), dtype=np.uint8) for _ in range(num_frames - len(frames))]
        frames += pad

    video = np.stack(frames, axis=0).astype(np.float32) / 255.0
    return video

def format_features_opencv(path, label):
    # Explicitly run tf.py_function on the CPU, ensure data types match.
    video = tf.numpy_function(read_video_opencv, [path], tf.float32)
    # Set the shape explicitly
    video.set_shape([num_frames, resolution, resolution, 3])
    label = tf.one_hot(label, num_classes)
    return video, label

def make_dataset(paths, labels, training=True):
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training:
        dataset = dataset.shuffle(100)
    # Apply dataset.map() outside the strategy.scope
    dataset = dataset.map(
        format_features_opencv,
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    # Add a tf.function decorator to format_features_opencv for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [7]:
train_dataset = make_dataset(train_paths, train_labels, training=True)
test_dataset = make_dataset(test_paths, test_labels, training=False)

In [8]:
!pip install tf-models-official==2.18.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=1e20e0a8cc03ec4f31802fdbd5582862c91ea55f0f5e883a7caa19f1e960fe27
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: numpy
    Found

In [9]:
# Run imports
#import tensorflow as tf
#from tf.keras import layers
#from tf.keras import optimizers
#import tensorflow_datasets as tfds

from official.vision.configs import video_classification
from official.projects.movinet.configs import movinet as movinet_configs
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_layers
from official.projects.movinet.modeling import movinet_model

In [10]:
model_id = 'a5'

tf.keras.backend.clear_session()

backbone = movinet.Movinet(model_id=model_id)
model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([1, 1, 1, 1, 3])

# Load pretrained weights
!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a5_base.tar.gz -O movinet_a5_base.tar.gz -q
!tar -xvf movinet_a5_base.tar.gz

checkpoint_dir = 'movinet_a5_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)

movinet_a5_base/
movinet_a5_base/checkpoint
movinet_a5_base/ckpt-1.data-00000-of-00001
movinet_a5_base/ckpt-1.index


In [11]:
def build_classifier(backbone, num_classes, freeze_backbone=False):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  if freeze_backbone:
    for layer in model.layers[:-1]:
      layer.trainable = False
    model.layers[-1].trainable = True

  return model

In [12]:
model = build_classifier(backbone, num_classes, freeze_backbone=True)

In [20]:
epochs = 3
train_steps = len(train_paths) // batch_size
total_train_steps = train_steps * epochs

loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)

metrics = [tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]

initial_learning_rate = 0.01
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=total_train_steps,
)

opt = tf.keras.optimizers.RMSprop(learning_rate, rho=0.9, momentum=0.9, epsilon=1.0, clipnorm=1.0)

model.compile(loss=loss_obj, optimizer=opt, metrics=metrics)

callbacks = [
    tf.keras.callbacks.TensorBoard(),
]

AttributeError: 'Variable' object has no attribute '_distribute_strategy'

In [ ]:
results = model.fit(
    train_dataset,
    epochs=num_epochs,  # Ajuste en fonction du nombre d'époques
    validation_data=test_dataset,
    verbose=2
)

In [19]:
# Define the distribution strategy
strategy = tf.distribute.MirroredStrategy()

# Create and compile the model within the strategy scope
with strategy.scope():
    model_id = 'a5'
    tf.keras.backend.clear_session()
    backbone = movinet.Movinet(model_id=model_id)

    # Build the classifier with the correct input shape and num_classes
    model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=num_classes)
    model.build([1, num_frames, resolution, resolution, 3]) # Correct input shape

    # Load pretrained weights (if needed)
    !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a5_base.tar.gz -O movinet_a5_base.tar.gz -q
    !tar -xvf movinet_a5_base.tar.gz
    checkpoint_dir = 'movinet_a5_base'
    checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
    checkpoint = tf.train.Checkpoint(model=model)

    def build_classifier(backbone, num_classes, freeze_backbone=False):
        """Builds a classifier on top of a backbone model."""
        model = movinet_model.MovinetClassifier(
            backbone=backbone,
            num_classes=num_classes)
        model.build([batch_size, num_frames, resolution, resolution, 3])

        if freeze_backbone:
            for layer in model.layers[:-1]:
                layer.trainable = False
            model.layers[-1].trainable = True

        return model

    # Build the model with the distribution strategy
    model = build_classifier(backbone, num_classes, freeze_backbone=True)

    # Compile the model within the strategy scope
    epochs = 3
    train_steps = len(train_paths) // batch_size
    total_train_steps = train_steps * epochs

    loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)

    metrics = [tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]

    initial_learning_rate = 0.01
    learning_rate = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate, decay_steps=total_train_steps,
    )

    opt = tf.keras.optimizers.RMSprop(learning_rate, rho=0.9, momentum=0.9, epsilon=1.0, clipnorm=1.0)

    model.compile(loss=loss_obj, optimizer=opt, metrics=metrics)

    callbacks = [
        tf.keras.callbacks.TensorBoard(),
    ]
# %%
results = model.fit(
    train_dataset,
    epochs=epochs,  # Ajuste en fonction du nombre d'époques
    validation_data=test_dataset,
    verbose=2
)

movinet_a5_base/
movinet_a5_base/checkpoint
movinet_a5_base/ckpt-1.data-00000-of-00001
movinet_a5_base/ckpt-1.index


IndexError: pop from empty list